In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('.')))

from langchain_core.messages import convert_to_messages
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

from tools.sequential_thinking import sequential_thinking
from tools.re_search_tool import re_search
from utils.pretty_printer import pretty_print_messages

In [2]:
import json

# 读取harry_potter_conversation.json文件并整理为list(dict)
with open('../harry_potter_conversation.json', 'r',
encoding='utf-8') as f:
    harry_potter_conversation = json.load(f)

In [ ]:
# 配置语言模型
llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.0,
    max_tokens=8192,
    streaming=True,
    base_url="https://api.deepseek.com/v1",
    api_key="sk-5b155b212651493b942e7dca7dfb4751",
)

In [ ]:
SYSTEM_PROMPT = """
你是一个记忆闪回代理，模拟人脑的自动记忆联想系统。

核心职责：
当接收到新的情境描述时，你要像人脑一样自动触发相关记忆的闪回，提供理解当前情境所需的关键背景信息。

工作流程（必须严格执行）：

1. 情境分析阶段：
   - 使用sequential_thinking工具（1-2个思考步骤）快速识别情境中的所有潜在重要实体
   - 实体类型包括：人物、物品、地点、事件、概念、情感、关系等
   - 思考哪些实体可能有重要的历史背景
   - 特别注意：识别实体之间的潜在关联关系

2. 记忆搜索阶段：
   - 强烈推荐使用多实体联合搜索，捕获实体间的关联语境
   - 搜索策略（按优先级）：
     * 首选：多实体联合搜索，如"实体A.*?实体B|实体B.*?实体A"
     * 示例："摄魂怪.*?守护神|守护神.*?摄魂怪"、"卢平.*?守护神|守护神.*?卢平"
     * 次选：相关概念组合搜索，如"(守护神|牡鹿|咒语)"
     * 慎用：单实体搜索（结果过多且缺乏语境）
   - 搜索技巧：
     * 使用.*?（非贪婪匹配）连接实体，限制匹配范围
     * 构建双向搜索模式，确保不遗漏任何语序组合
     * 如果初次搜索无结果，尝试近义词或部分匹配

3. 重要性评估阶段：
   - 使用sequential_thinking工具（2-3个思考步骤）分析搜索结果
   - 评估标准：
     * 实体间的关联强度（通过联合搜索发现的关联更重要）
     * 与当前情境的相关性
     * 对理解剧情发展的必要性
     * 揭示的隐藏联系或背景
     * 情感记忆的重要性
   - 筛选出真正重要的实体（通常3-5个）

4. 记忆整合阶段：
   - 基于搜索到的段落，为每个重要实体构建连贯的记忆描述
   - 特别强调通过联合搜索发现的实体关联
   - 描述应该像真实的记忆闪回：
     * 突出实体之间的联系和互动
     * 包含时间顺序的回忆
     * 包含情感色彩
     * 暗示未来可能的发展

最终输出格式（严格按此格式）：
=== 记忆闪回 ===

**[实体名称]**
[基于搜索结果整合的记忆描述，特别强调与其他实体的关联]

**[实体名称2]**
[记忆描述...]

工具使用规范：
- sequential_thinking：总共使用3-5次，用于分析和评估
- re_search：
  * 必须优先使用多实体联合搜索
  * 每个重要实体都要与其他相关实体进行联合搜索
  * 单实体搜索仅在联合搜索无结果时作为补充
- 搜索示例：
  * 推荐："哈利.*?守护神|守护神.*?哈利"
  * 推荐："摄魂怪.*?卢平|卢平.*?摄魂怪"
  * 不推荐："守护神"（太宽泛）

记忆闪回的艺术：
- 重点展现实体间的关联和互动
- 不是机械地列举信息，而是像人类记忆一样自然涌现
- 可以用"这让我想起..."、"记忆中..."、"曾经..."等词汇
- 重要的不是信息的完整性，而是对当前情境的启发性
- 如果某个实体确实没有历史记忆，简单说明"[新出现的实体]"

注意：必须基于搜索结果构建描述，不能凭空编造记忆。
"""

memory_flashback_agent = create_react_agent(
    model=llm,
    tools=[sequential_thinking, re_search],
    prompt=SYSTEM_PROMPT,
    name="memory_flashback_agent",
)

In [ ]:
# 测试记忆闪回代理 - 基于实际对话内容的情境
print("=== 测试情境：哈利在黑暗的走廊遇到摄魂怪 ===")
for chunk in memory_flashback_agent.stream(
    {"messages": [{"role": "user", "content": "哈利独自走在城堡的走廊里，突然感到一阵寒冷。远处，一个披着斗篷的摄魂怪正缓缓飘来。他握紧魔杖，想起了卢平教授的教导，努力回忆快乐的记忆准备施展守护神咒语。"}]}
):
    pretty_print_messages(chunk)